In [69]:
%run main.py

C:\Users\peyrucha\GitHub_W\daily-arxiv-noti\main.py:21: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 21 of the file c:\Users\peyrucha\GitHub_W\daily-arxiv-noti\main.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = bs(page)


i= 0 
 dt_list[i]= 
[1]

        arXiv:2405.12254
      
      
        [pdf, ps, other]
    
paper title is: Three fold Spin valley intertwined Dirac cone in nonmagnetic Weyl Semimetal Pt$_3$Sn$_2$S$_2$ with the Kagome lattice A theoretical DFT perspective
i= 1 
 dt_list[i]= 
[2]

        arXiv:2405.12278
      
      
        [pdf, ps, other]
    
paper title is: Emergent Majorana metal from a chiral spin liquid
i= 2 
 dt_list[i]= 
[3]

        arXiv:2405.12279
      
      
        [pdf, ps, other]
    
paper title is: Eigenstate localization in a many-body quantum system
i= 3 
 dt_list[i]= 
[4]

        arXiv:2405.12292
      
      
        [pdf, ps, other]
    
paper title is: Theory of fractional quantum Hall liquids coupled to quantum light and emergent graviton-polaritons
i= 4 
 dt_list[i]= 
[5]

        arXiv:2405.12294
      
      
        [pdf, ps, other]
    
paper title is: Stabilizing fractional Chern insulators via exchange interaction in moir\'e systems
i= 5 
 dt_list

In [59]:
# encoding: utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from bs4 import BeautifulSoup as bs
import urllib.request
import numpy as np
from github_issue import make_github_issue
from config import NEW_SUB_URL, KEYWORD_LIST


In [4]:
URL = ['https://arxiv.org/catchup?smonth=5&group=grp_&sday=6&num=2000&archive=quant-ph&method=with&syear=2024']

In [41]:
len(dt_list), 

(1457, 8)

In [45]:
import time

In [68]:
dd_list[0].find("div", {"class": "list-title mathjax"}).text.replace("Title:", "").strip()

'Dynamical suppression of many-body non-Hermitian skin effect in Anyonic systems'

In [70]:
## URL of custom search for missing dates

# NEW_SUB_URL = ['https://arxiv.org/catchup?smonth=5&group=grp_&sday=6&num=2000&archive=quant-ph&method=with&syear=2024']
NEW_SUB_URL = ['https://arxiv.org/catchup?smonth=5&group=grp_&sday=6&num=2000&archive=cond-mat&method=with&syear=2024']


keyword_list = KEYWORD_LIST
keyword_dict = {key: [] for key in keyword_list}

for sub_url in NEW_SUB_URL:
    print(sub_url)
    page = urllib.request.urlopen(sub_url)
    soup = bs(page)
    content = soup.body.find("div", {'id': 'content'})

    issue_title = content.find("h3").text
    
    ## initial version
    dt_list = content.dl.find_all("dt")
    dd_list = content.dl.find_all("dd")
    ## corrected version for catchup
    dt_list = content.find_all("dt")
    dd_list = content.find_all("dd")

    arxiv_base = "https://arxiv.org/abs/"

    assert len(dt_list) == len(dd_list)

    ## split issue into multiple issues if there are more than 200 papers
    N_max = 200
    for jj in range( len(dt_list)//N_max+1 ):
        paper_dict = {} #key=paper_number, content=url,number,abstract,keywords,authors,..

        for i in range(jj*N_max, (jj+1)*N_max):
            if i>=len(dt_list): break
            
            # print('i=',i,'\n dt_list[i]=',dt_list[i].text)
            paper = {}
            # paper_number = dt_list[i].text.strip().split(" ")[2].split(":")[-1] ##old version
            # paper_number = dt_list[i].text.strip().split('\n')[2].strip() ## corrected version for daily arxiv
            paper_number = dt_list[i].text.strip().split('[')[0].strip() ## corrected version for catchup

            # paper_number = dt_list[i].text.strip().split(" ")[0].split(":")[-1] ##UNCOMMENT FOR CUSTOM DATE ARXIV SERACH
            paper['main_page'] = arxiv_base + paper_number
            paper['pdf'] = arxiv_base.replace('abs', 'pdf') + paper_number

            paper['title'] = dd_list[i].find("div", {"class": "list-title mathjax"}).text.replace("Title: ", "").strip()
            paper['authors'] = dd_list[i].find("div", {"class": "list-authors"}).text.replace("Authors:\n", "").replace(
                "\n", "").strip()
            paper['subjects'] = dd_list[i].find("div", {"class": "list-subjects"}).text.replace("Subjects: ", "").strip()
            paper['abstract'] = dd_list[i].find("p", {"class": "mathjax"}).text.replace("\n", " ").strip()

            #check if keywords are in paper
            for keyword in keyword_list:
                if keyword.lower() in paper['abstract'].lower() or keyword.lower() in paper['title'].lower():
                    # print('keyword found:',keyword,'in paper:',paper['main_page'])
                    #if it is, add paper to paper_dict if not already added. Otherwise, just add new keyword to paper entry
                    ## first check if paper is not added yet
                    if paper['main_page'] not in paper_dict:
                        # if its a new paper, add it to dict
                        paper['keyword']=[keyword]
                        # print('new paper found with keyword',paper['keyword'])
                        paper_dict[paper['main_page']] = paper
                    else:
                        # print('already added paper with second keyword',keyword)
                        paper_dict[paper['main_page']]['keyword'] = paper_dict[paper['main_page']]['keyword'] + [keyword]
                        # print('new keyword for this paper is:', paper_dict[paper['main_page']]['keyword'])
                    
        full_report = ''
        for paper in paper_dict.values():
            # print('first paper in paper_dict is\n',paper)
            full_report = full_report + f'## {paper["title"]}\n'
            full_report = full_report + '- **Keywords:** {}\n - **Authors:** {}\n - **Subjects:** {}\n - **Arxiv link:** {}\n - **Pdf link:** {}\n - **Abstract**\n {}\n' \
                    .format(paper['keyword'], paper['authors'], paper['subjects'], paper['main_page'], paper['pdf'],paper['abstract'])

        time.sleep(5)

        # for keyword in keyword_list:
        #     full_report = full_report + '## Keyword: ' + keyword + '\n'

        #     if len(keyword_dict[keyword]) == 0:
        #         full_report = full_report + 'There is no result \n'

        #     for paper in keyword_dict[keyword]:
        #         report = '### {}\n - **Authors:** {}\n - **Subjects:** {}\n - **Arxiv link:** {}\n - **Pdf link:** {}\n - **Abstract**\n {}' \
        #             .format(paper['title'], paper['authors'], paper['subjects'], paper['main_page'], paper['pdf'],
        #                     paper['abstract'])
        #         full_report = full_report + report + '\n'

        print(full_report[:500])
        make_github_issue(title=issue_title, body=full_report, labels=keyword_list)


https://arxiv.org/catchup?smonth=5&group=grp_&sday=6&num=2000&archive=cond-mat&method=with&syear=2024
## Superconductivity of Bulk Abnormal Magic-stoichiometric Na3Cl Salt  Crystals at Normal Pressure
- **Keywords:** ['superconducting']
 - **Authors:** Shuqiang He, Yi-Feng Zheng, Guosheng Shi, Yi-Jie Xiang, Meihui Xiao, Qituan Zhang, Yue-Yu Zhang, Haiping Fang
 - **Subjects:** Superconductivity (cond-mat.supr-con)
 - **Arxiv link:** https://arxiv.org/abs/arXiv:2405.01570
 - **Pdf link:** https://arxiv.org/pdf/arXiv:2405.01570
 - **Abstract**
 The identification of new materials with superconduct
Successfully created Issue "Replacements"
## Anisotropic transport properties in prismatic topological insulator  nanowires
- **Keywords:** ['topological']
 - **Authors:** Hallmann Oskar Gestsson, Andrei Manolescu, Jens H. Bardarson, Sigurdur I. Erlingsson
 - **Subjects:** Mesoscale and Nanoscale Physics (cond-mat.mes-hall)
 - **Arxiv link:** https://arxiv.org/abs/arXiv:2405.03380
 - **Pdf link

"## Improving Trainability of Variational Quantum Circuits via  Regularization Strategies\n- **Keywords:** ['circuit']\n - **Authors:** Jun Zhuang, Jack Cunningham, Chaowen Guan\n - **Subjects:** Quantum Physics (quant-ph); Machine Learning (cs.LG)\n - **Arxiv link:** https://arxiv.org/abs/arXiv:2405.01606\n - **Pdf link:** https://arxiv.org/pdf/arXiv:2405.01606\n - **Abstract**\n In the era of noisy intermediate-scale quantum (NISQ), variational quantum circuits (VQCs) have been widely applied in vario"

In [36]:
print(full_report[:2000])
len(full_report.split("##"))
print('\n\n\n')
print(full_report[-2000:])

## Improving Trainability of Variational Quantum Circuits via  Regularization Strategies
- **Keywords:** ['circuit']
 - **Authors:** Jun Zhuang, Jack Cunningham, Chaowen Guan
 - **Subjects:** Quantum Physics (quant-ph); Machine Learning (cs.LG)
 - **Arxiv link:** https://arxiv.org/abs/arXiv:2405.01606
 - **Pdf link:** https://arxiv.org/pdf/arXiv:2405.01606
 - **Abstract**
 In the era of noisy intermediate-scale quantum (NISQ), variational quantum circuits (VQCs) have been widely applied in various domains, advancing the superiority of quantum circuits against classic models. Similar to classic models, regular VQCs can be optimized by various gradient-based methods. However, the optimization may be initially trapped in barren plateaus or eventually entangled in saddle points during training. These gradient issues can significantly undermine the trainability of VQC. In this work, we propose a strategy that regularizes model parameters with prior knowledge of the train data and Gaussian n

In [24]:
paper

{'main_page': 'https://arxiv.org/abs/arXiv:2405.01606',
 'pdf': 'https://arxiv.org/pdf/arXiv:2405.01606',
 'title': 'Improving Trainability of Variational Quantum Circuits via  Regularization Strategies',
 'authors': 'Jun Zhuang, Jack Cunningham, Chaowen Guan',
 'subjects': 'Quantum Physics (quant-ph); Machine Learning (cs.LG)',
 'abstract': 'In the era of noisy intermediate-scale quantum (NISQ), variational quantum circuits (VQCs) have been widely applied in various domains, advancing the superiority of quantum circuits against classic models. Similar to classic models, regular VQCs can be optimized by various gradient-based methods. However, the optimization may be initially trapped in barren plateaus or eventually entangled in saddle points during training. These gradient issues can significantly undermine the trainability of VQC. In this work, we propose a strategy that regularizes model parameters with prior knowledge of the train data and Gaussian noise diffusion. We conduct abla

In [20]:
len(dt_list)

1457

In [21]:
len(dd_list)

1457

In [22]:
dd_list[4]

<dd>
<div class="meta">
<div class="list-title mathjax">
<span class="descriptor">Title:</span> Floquet engineered inhomogeneous quantum chaos in critical systems
</div>
<div class="list-authors">
<span class="descriptor">Authors:</span>
<a href="/search/cond-mat?searchtype=author&amp;query=Lapierre%2C+B">Bastien Lapierre</a>, 
<a href="/search/cond-mat?searchtype=author&amp;query=Numasawa%2C+T">Tokiro Numasawa</a>, 
<a href="/search/cond-mat?searchtype=author&amp;query=Neupert%2C+T">Titus Neupert</a>, 
<a href="/search/cond-mat?searchtype=author&amp;query=Ryu%2C+S">Shinsei Ryu</a>
</div>
<div class="list-subjects">
<span class="descriptor">Subjects:</span> <span class="primary-subject">Strongly Correlated Electrons (cond-mat.str-el)</span>; High Energy Physics - Theory (hep-th); Quantum Physics (quant-ph)

</div>
<p class="mathjax">We study universal chaotic dynamics of a large class of periodically driven
critical systems described by spatially inhomogeneous conformal field theories.

In [23]:
paper_dict

{'https://arxiv.org/abs/arXiv:2405.01606': {'main_page': 'https://arxiv.org/abs/arXiv:2405.01606',
  'pdf': 'https://arxiv.org/pdf/arXiv:2405.01606',
  'title': 'Improving Trainability of Variational Quantum Circuits via  Regularization Strategies',
  'authors': 'Jun Zhuang, Jack Cunningham, Chaowen Guan',
  'subjects': 'Quantum Physics (quant-ph); Machine Learning (cs.LG)',
  'abstract': 'In the era of noisy intermediate-scale quantum (NISQ), variational quantum circuits (VQCs) have been widely applied in various domains, advancing the superiority of quantum circuits against classic models. Similar to classic models, regular VQCs can be optimized by various gradient-based methods. However, the optimization may be initially trapped in barren plateaus or eventually entangled in saddle points during training. These gradient issues can significantly undermine the trainability of VQC. In this work, we propose a strategy that regularizes model parameters with prior knowledge of the train da

In [25]:
len(dt_list)

235

In [22]:
content.dl

<dl>
<dt> <span class="list-identifier"><a href="/abs/2405.01606" title="Abstract">arXiv:2405.01606</a> [<a href="/pdf/2405.01606" title="Download PDF">pdf</a>, <a href="/format/2405.01606" title="Other formats">other</a>]</span></dt>
<dd>
<div class="meta">
<div class="list-title mathjax">
<span class="descriptor">Title:</span> Improving Trainability of Variational Quantum Circuits via  Regularization Strategies
</div>
<div class="list-authors">
<span class="descriptor">Authors:</span>
<a href="/search/quant-ph?searchtype=author&amp;query=Zhuang%2C+J">Jun Zhuang</a>, 
<a href="/search/quant-ph?searchtype=author&amp;query=Cunningham%2C+J">Jack Cunningham</a>, 
<a href="/search/quant-ph?searchtype=author&amp;query=Guan%2C+C">Chaowen Guan</a>
</div>
<div class="list-comments mathjax">
<span class="descriptor">Comments:</span> preprint, under review. TL;DR: we propose a regularization strategy to improve the trainability of VQCs
</div>
<div class="list-subjects">
<span class="descriptor">

In [21]:
content.dl.find_all("dt")

[<dt> <span class="list-identifier"><a href="/abs/2405.01606" title="Abstract">arXiv:2405.01606</a> [<a href="/pdf/2405.01606" title="Download PDF">pdf</a>, <a href="/format/2405.01606" title="Other formats">other</a>]</span></dt>,
 <dt> <span class="list-identifier"><a href="/abs/2405.01622" title="Abstract">arXiv:2405.01622</a> [<a href="/pdf/2405.01622" title="Download PDF">pdf</a>, <a href="/format/2405.01622" title="Other formats">other</a>]</span></dt>,
 <dt> <span class="list-identifier"><a href="/abs/2405.01640" title="Abstract">arXiv:2405.01640</a> [<a href="/pdf/2405.01640" title="Download PDF">pdf</a>, <a href="/format/2405.01640" title="Other formats">other</a>]</span></dt>,
 <dt> <span class="list-identifier"><a href="/abs/2405.01641" title="Abstract">arXiv:2405.01641</a> [<a href="/pdf/2405.01641" title="Download PDF">pdf</a>, <a href="/format/2405.01641" title="Other formats">other</a>]</span></dt>,
 <dt> <span class="list-identifier"><a href="/abs/2405.01642" title="Abs

In [17]:
len(dt_list)

46

In [19]:
for x in dt_list:
    print(x) 

<dt> <span class="list-identifier"><a href="/abs/2405.01606" title="Abstract">arXiv:2405.01606</a> [<a href="/pdf/2405.01606" title="Download PDF">pdf</a>, <a href="/format/2405.01606" title="Other formats">other</a>]</span></dt>
<dt> <span class="list-identifier"><a href="/abs/2405.01622" title="Abstract">arXiv:2405.01622</a> [<a href="/pdf/2405.01622" title="Download PDF">pdf</a>, <a href="/format/2405.01622" title="Other formats">other</a>]</span></dt>
<dt> <span class="list-identifier"><a href="/abs/2405.01640" title="Abstract">arXiv:2405.01640</a> [<a href="/pdf/2405.01640" title="Download PDF">pdf</a>, <a href="/format/2405.01640" title="Other formats">other</a>]</span></dt>
<dt> <span class="list-identifier"><a href="/abs/2405.01641" title="Abstract">arXiv:2405.01641</a> [<a href="/pdf/2405.01641" title="Download PDF">pdf</a>, <a href="/format/2405.01641" title="Other formats">other</a>]</span></dt>
<dt> <span class="list-identifier"><a href="/abs/2405.01642" title="Abstract">ar

'arXiv:2405.01606'

In [10]:
dd_list[0]

<dd>
<div class="meta">
<div class="list-title mathjax">
<span class="descriptor">Title:</span> Improving Trainability of Variational Quantum Circuits via  Regularization Strategies
</div>
<div class="list-authors">
<span class="descriptor">Authors:</span>
<a href="/search/quant-ph?searchtype=author&amp;query=Zhuang%2C+J">Jun Zhuang</a>, 
<a href="/search/quant-ph?searchtype=author&amp;query=Cunningham%2C+J">Jack Cunningham</a>, 
<a href="/search/quant-ph?searchtype=author&amp;query=Guan%2C+C">Chaowen Guan</a>
</div>
<div class="list-comments mathjax">
<span class="descriptor">Comments:</span> preprint, under review. TL;DR: we propose a regularization strategy to improve the trainability of VQCs
</div>
<div class="list-subjects">
<span class="descriptor">Subjects:</span> <span class="primary-subject">Quantum Physics (quant-ph)</span>; Machine Learning (cs.LG)

</div>
<p class="mathjax">In the era of noisy intermediate-scale quantum (NISQ), variational quantum
circuits (VQCs) have been 

In [78]:
# soup.body.find("div", {'id': 'content'})

In [79]:
# soup.body

In [31]:
        paper_number = dt_list[i].text.strip().split(" ")[2].split(":")[-1]


In [45]:
dt_list[20].text.strip()

'[21]\n\n        arXiv:2405.09881\n      \n      \n        [pdf, ps, html, other]'

In [53]:
dt_list[20].text.strip().split('\n')[2].strip()

'arXiv:2405.09881'

In [22]:
paper_number

''

In [18]:
keyword

'transmon'

In [20]:
paper_dict

{'https://arxiv.org/abs/': {'main_page': 'https://arxiv.org/abs/',
  'pdf': 'https://arxiv.org/pdf/',
  'title': 'Title:\n          Fermionic quantum criticality through the lens of topological holography',
  'authors': 'Sheng-Jie Huang',
  'subjects': 'Subjects:\nStrongly Correlated Electrons (cond-mat.str-el); High Energy Physics - Theory (hep-th); Mathematical Physics (math-ph)',
  'abstract': 'We utilize the topological holographic framework to characterize and gain insights into the nature of quantum critical points and gapless phases in fermionic quantum systems. Topological holography is a general framework that describes the generalized global symmetry and the symmetry charges of a local quantum system in terms of a slab of a topological order, termed as the symmetry topological field theory (SymTFT), in one higher dimension. In this work, we consider a generalization of the topological holographic picture for $(1+1)d$ fermionic quantum phases of matter. We discuss how spin str

In [8]:
issue_title

'New submissions for Friday, 17 May 2024 (showing 45 of 45 entries )'

In [12]:
dt_list[0]

<dt>
<a name="item1">[1]</a>
<a href="/abs/2405.09613" id="2405.09613" title="Abstract">
        arXiv:2405.09613
      </a>
      
        [<a aria-labelledby="pdf-2405.09613" href="/pdf/2405.09613" id="pdf-2405.09613" title="Download PDF">pdf</a>, <a aria-labelledby="ps-2405.09613" href="/ps/2405.09613" id="ps-2405.09613" title="Download PostScript">ps</a>, <a aria-labelledby="html-2405.09613" href="https://arxiv.org/html/2405.09613v1" id="html-2405.09613" rel="noopener noreferrer" target="_blank" title="View HTML">html</a>, <a aria-labelledby="oth-2405.09613" href="/format/2405.09613" id="oth-2405.09613" title="Other formats">other</a>]
    </dt>

In [13]:
dd_list[0]

<dd>
<div class="meta">
<div class="list-title mathjax"><span class="descriptor">Title:</span>
          Computable entanglement cost
        </div>
<div class="list-authors"><a href="https://arxiv.org/search/quant-ph?searchtype=author&amp;query=Lami,+L">Ludovico Lami</a>, <a href="https://arxiv.org/search/quant-ph?searchtype=author&amp;query=Mele,+F+A">Francesco Anna Mele</a>, <a href="https://arxiv.org/search/quant-ph?searchtype=author&amp;query=Regula,+B">Bartosz Regula</a></div>
<div class="list-comments mathjax"><span class="descriptor">Comments:</span>
          7+24 pages, no figures
        </div>
<div class="list-subjects"><span class="descriptor">Subjects:</span>
<span class="primary-subject">Quantum Physics (quant-ph)</span>; Statistical Mechanics (cond-mat.stat-mech); Mathematical Physics (math-ph)
        </div>
<p class="mathjax">
          Quantum information theory is plagued by the problem of regularisations, which require the evaluation of formidable asymptotic quanti